# Automatic Differentiation

This code is from https://huggingface.co/blog/andmholm/what-is-automatic-differentiation

In [3]:
import numpy as np


class Variable:

    def __init__(self, primal, adjoint=0.0):
        self.primal = primal
        self.adjoint = adjoint

    def backward(self, adjoint):
        self.adjoint += adjoint

    def __add__(self, other):
        variable = Variable(self.primal + other.primal)

        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint * 1.0
            other_adjoint = adjoint * 1.0
            self.backward(self_adjoint)
            other.backward(other_adjoint)

        variable.backward = backward
        return variable

    def __sub__(self, other):
        variable = Variable(self.primal - other.primal)

        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint * 1.0
            other_adjoint = adjoint * -1.0
            self.backward(self_adjoint)
            other.backward(other_adjoint)

        variable.backward = backward
        return variable

    def __mul__(self, other):
        variable = Variable(self.primal * other.primal)

        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint * other.primal
            other_adjoint = adjoint * self.primal
            self.backward(self_adjoint)
            other.backward(other_adjoint)

        variable.backward = backward
        return variable

    def __truediv__(self, other):
        variable = Variable(self.primal / other.primal)

        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint * (1.0 / other.primal)
            other_adjoint = adjoint * (-1.0 * self.primal / other.primal**2)
            self.backward(self_adjoint)
            other.backward(other_adjoint)

        variable.backward = backward
        return variable

    def __repr__(self) -> str:
        return f"primal: {self.primal}, adjoint: {self.adjoint}"

def sin(x):
    variable = Variable(np.sin(x.primal))

    def backward(adjoint):
        variable.adjoint += adjoint * np.cos(x.primal)
        x_adjoint = adjoint * np.cos(x.primal)
        x.backward(x_adjoint)

    variable.backward = backward
    return variable

def cos(x):
    variable = Variable(np.cos(x.primal))

    def backward(adjoint):
        variable.adjoint += adjoint * -np.sin(x.primal)
        x_adjoint = adjoint * -np.sin(x.primal)
        x.backward(x_adjoint)

    variable.backward = backward
    return variable

def ln(x):
    variable = Variable(np.log(x.primal))

    def backward(adjoint):
        variable.adjoint += adjoint * (1.0 / x.primal)
        x_adjoint = adjoint * (1.0 / x.primal)
        x.backward(x_adjoint)

    variable.backward = backward
    return variable


In [6]:
x1 = Variable(2.0)
x2 = Variable(5.0)

y = x1 * cos(x2) + ln(x1)
print(y)

y.backward(1.0)
print(x1)
print(x2)

primal: 1.2604715514863978, adjoint: 0.0
primal: 2.0, adjoint: 0.7836621854632262
primal: 5.0, adjoint: 1.917848549326277


# Adapting for Vectors and Matrices

In [3]:
import numpy as np

class Variable:
    def __init__(self, primal, adjoint=None):
        self.primal = np.array(primal)
        self.adjoint = np.zeros_like(self.primal) if adjoint is None else np.array(adjoint)

    def backward(self, adjoint):
        self.adjoint += adjoint

    def __add__(self, other):
        variable = Variable(self.primal + other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint
            other_adjoint = adjoint
            self.backward(self_adjoint)
            other.backward(other_adjoint)
        variable.backward = backward
        return variable

    def __sub__(self, other):
        variable = Variable(self.primal - other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint
            other_adjoint = -adjoint
            self.backward(self_adjoint)
            other.backward(other_adjoint)
        variable.backward = backward
        return variable

    def __mul__(self, other):
        variable = Variable(self.primal * other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint * other.primal
            other_adjoint = adjoint * self.primal
            self.backward(self_adjoint)
            other.backward(other_adjoint)
        variable.backward = backward
        return variable

    def __matmul__(self, other):
        variable = Variable(np.matmul(self.primal, other.primal))
        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = np.matmul(adjoint, other.primal.T)
            other_adjoint = np.matmul(self.primal.T, adjoint)
            self.backward(self_adjoint)
            other.backward(other_adjoint)
        variable.backward = backward
        return variable

    def __truediv__(self, other):
        variable = Variable(self.primal / other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self_adjoint = adjoint / other.primal
            other_adjoint = -adjoint * self.primal / (other.primal ** 2)
            self.backward(self_adjoint)
            other.backward(other_adjoint)
        variable.backward = backward
        return variable

    def __repr__(self) -> str:
        return f"primal:\n{self.primal}\nadjoint:\n{self.adjoint}"

In [ ]:
# Scalar operations
a = Variable(2.0)
b = Variable(3.0)
c = a * b
c.backward(1.0)
print(a, b)

# Vector operations
v1 = Variable([1, 2, 3])
v2 = Variable([4, 5, 6])
v3 = v1 + v2
v3.backward(np.array([1, 1, 1]))
print(v1, v2)

# Matrix operations
m1 = Variable([[1, 2], [3, 4]])
m2 = Variable([[5, 6], [7, 8]])
m3 = m1 @ m2  # Matrix multiplication
m3.backward(np.array([[1, 1], [1, 1]]))
print(m1, m2)

In [ ]:
import torch

# Example from paper
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(5.0, requires_grad=True)
c = torch.log(a) + a * b - torch.sin(b)
print(c)
c.backward()
print(a.grad, b.grad)

# Custom example
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(5.0, requires_grad=True)
c = a * torch.cos(b) + torch.log(a)
print(c)
c.backward()
print(a.grad, b.grad)